In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import requests, os, re, urllib.parse as up
from bs4 import BeautifulSoup

In [62]:
url = "https://camara.cl/sala/default.aspx#!/verTabla/4178/684/null/Default/0.1408008372422508"

opts = webdriver.ChromeOptions()
opts.add_argument("--start-maximized")

# la forma correcta:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
)
driver.get(url)

wait = WebDriverWait(driver, 15)

In [64]:
titulo = "INFORMACIÓN Y DOCUMENTOS DE LA SESIÓN"
btn = wait.until(EC.element_to_be_clickable((
    By.XPATH,
    "//*[self::button or self::a or self::div][normalize-space()='%s' or .//span[normalize-space()='%s']]" % (titulo, titulo)
)))
driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
btn.click()

In [65]:
boletin_btn = wait.until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, "div.item-button.button-boletin")
))

In [66]:
cls = boletin_btn.get_attribute("class") or ""
is_disabled = "button-disabled" in cls or (boletin_btn.get_attribute("data-ng-disabled") in ("true","True"))

In [67]:

if is_disabled:
    raise RuntimeError("El botón 'Boletín de sesión' está deshabilitado en esta sesión (BoletinId == -1).")

# 3) Intentar click normal; si no funciona, invocar Angular directamente
try:
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", boletin_btn)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.item-button.button-boletin")))
    boletin_btn.click()
except Exception:
    # Llamada directa a la función Angular: defaultController.showDocSesion(defaultController.SesionSelected.BoletinId)
    js = """
    var el = arguments[0];
    var ngEl = angular.element(el);
    var scope = ngEl.scope() || ngEl.isolateScope();
    if(!scope) return "no-scope";
    var ctrl = scope.defaultController || scope.$parent && scope.$parent.defaultController;
    if(!ctrl) return "no-ctrl";
    var id = ctrl.SesionSelected && ctrl.SesionSelected.BoletinId;
    if(!id || id === -1) return "no-id";
    ctrl.showDocSesion(id);
    return "ok";
    """
    res = driver.execute_script(js, boletin_btn)
    if res not in ("ok",):
        raise RuntimeError(f"No se pudo invocar showDocSesion() (resultado JS: {res})")

In [68]:
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
src = iframe.get_attribute("src")
print("Iframe src:", src)

Iframe src: https://camara.cl/sala/Scripts/PDFJs/web/viewer.html?file=..%2F..%2F..%2FdoGet.asmx%2FgetDocumentoSesion%3FprmDocumentoId%3D15226%26noCache%3D0.7472785746437006


In [69]:
parsed = up.urlparse(src)
qs = up.parse_qs(parsed.query)
file_param = qs.get("file", [None])[0]

In [71]:
if not file_param:
    raise RuntimeError("No encontré el parámetro file= en el iframe src")

pdf_url = up.unquote(file_param)  
print("PDF URL:", pdf_url)

# 3) Descargar con cookies de Selenium (por si son necesarias)
sess = requests.Session()
for c in driver.get_cookies():
    sess.cookies.set(c["name"], c["value"], domain=c.get("domain"), path=c.get("path","/"))
headers = {"User-Agent": "Mozilla/5.0"}
r = sess.get(file_param, headers=headers, timeout=45)

# 4) Guardar en disco
os.makedirs("pdfs", exist_ok=True)
fname = "BoletinSesion.pdf"
cd = r.headers.get("content-disposition", "")
m = re.search(r'filename="?([^"]+)"?', cd)
if m:
    fname = m.group(1)
fname = re.sub(r'[\\/:*?"<>|]+', "_", fname)

with open(os.path.join("pdfs", fname), "wb") as f:
    f.write(r.content)

print("Descargado en: pdfs/" + fname)

PDF URL: ../../../doGet.asmx/getDocumentoSesion?prmDocumentoId=15226&noCache=0.7472785746437006


MissingSchema: Invalid URL '../../../doGet.asmx/getDocumentoSesion?prmDocumentoId=15226&noCache=0.7472785746437006': No scheme supplied. Perhaps you meant https://../../../doGet.asmx/getDocumentoSesion?prmDocumentoId=15226&noCache=0.7472785746437006?

In [58]:
driver.quit()